<a href="https://colab.research.google.com/github/INLANA01/capstone_project_coref_with_transformers/blob/main/BERT_%2B_Neural_Network_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import necessary libraries

In [ ]:
import os
import gc
import numpy as np
import pandas as pd 
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from tqdm import tqdm, trange
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset, Dataset)

# Some of the libraries in extract_features.py use Tensorflow Version 1, hence need to install older version of TenforFlow and disable v2 behaviour

In [ ]:
!pip install tensorflow==1.15
!pip install tensorflow-gpu==1.15
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

     |████████████████████████████████| 412.3MB 33kB/s 
     |████████████████████████████████| 512kB 46.3MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 3.8MB 48.7MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=0e6d077fc98c675e064731336be3f6e7bca5527e360c66d081ef352c711c3e11
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninsta

## Get the files withour cloning the gitrepo https://stackoverflow.com/questions/4604663/download-single-files-from-github

In [ ]:
!wget -q https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-development.tsv
!wget -q https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-validation.tsv
!wget -q https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-test.tsv
!wget -q https://raw.githubusercontent.com/google-research/bert/master/modeling.py 
!wget -q https://raw.githubusercontent.com/google-research/bert/master/extract_features.py 
!wget -q https://raw.githubusercontent.com/google-research/bert/master/tokenization.py

In [ ]:
import modeling
import extract_features
import tokenization

# Load the dataset into dataframe using Pandas

In [ ]:
val_df = pd.read_table('gap-validation.tsv', index_col='ID').reset_index(drop=True)
test_df  = pd.read_table('gap-validation.tsv', index_col='ID').reset_index(drop=True)
dev_df  = pd.read_table('gap-development.tsv', index_col='ID').reset_index(drop=True)

# Download BERT model, to extract features and get embeddings

In [ ]:
!wget -q https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip 

# Extract Zip file

In [ ]:
!unzip uncased_L-12_H-768_A-12.zip

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 8.3MB/s 
     |████████████████████████████████| 901kB 54.9MB/s 
     |████████████████████████████████| 3.3MB 55.6MB/s 


In [ ]:
from transformers import BertTokenizer, BertForMultipleChoice
import torch

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import codecs
import collections
import json
import re

import modeling
import tokenization
import tensorflow as tf

In [ ]:
class InputExample(object):

  def __init__(self, unique_id, text_a, text_b):
    self.unique_id = unique_id
    self.text_a = text_a
    self.text_b = text_b


class InputFeatures(object):
  """A single set of features of data."""

  def __init__(self, unique_id, tokens, input_ids, input_mask, input_type_ids):
    self.unique_id = unique_id
    self.tokens = tokens
    self.input_ids = input_ids
    self.input_mask = input_mask
    self.input_type_ids = input_type_ids


def input_fn_builder(features, seq_length):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_unique_ids = []
  all_input_ids = []
  all_input_mask = []
  all_input_type_ids = []

  for feature in features:
    all_unique_ids.append(feature.unique_id)
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_input_type_ids.append(feature.input_type_ids)

  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]

    num_examples = len(features)

    # This is for demo purposes and does NOT scale to large data sets. We do
    # not use Dataset.from_generator() because that uses tf.py_func which is
    # not TPU compatible. The right way to load data is with TFRecordReader.
    d = tf.data.Dataset.from_tensor_slices({
        "unique_ids":
            tf.constant(all_unique_ids, shape=[num_examples], dtype=tf.int32),
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_type_ids":
            tf.constant(
                all_input_type_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
    })

    d = d.batch(batch_size=batch_size, drop_remainder=False)
    return d

  return input_fn


def model_fn_builder(bert_config, init_checkpoint, layer_indexes, use_tpu,
                     use_one_hot_embeddings):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    unique_ids = features["unique_ids"]
    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    input_type_ids = features["input_type_ids"]

    model = modeling.AlbertModel(
        config=bert_config,
        is_training=False,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=input_type_ids,
        use_one_hot_embeddings=use_one_hot_embeddings)

    if mode != tf.estimator.ModeKeys.PREDICT:
      raise ValueError("Only PREDICT modes are supported: %s" % (mode))

    tvars = tf.trainable_variables()
    scaffold_fn = None
    (assignment_map,
     initialized_variable_names) = modeling.get_assignment_map_from_checkpoint(
         tvars, init_checkpoint)
    if use_tpu:

      def tpu_scaffold():
        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
        return tf.train.Scaffold()

      scaffold_fn = tpu_scaffold
    else:
      tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

    tf.logging.info("**** Trainable Variables ****")
    for var in tvars:
      init_string = ""
      if var.name in initialized_variable_names:
        init_string = ", *INIT_FROM_CKPT*"
      tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                      init_string)

    all_layers = model.get_all_encoder_layers()

    predictions = {
        "unique_id": unique_ids,
    }

    for (i, layer_index) in enumerate(layer_indexes):
      predictions["layer_output_%d" % i] = all_layers[layer_index]

    output_spec = tf.contrib.tpu.TPUEstimatorSpec(
        mode=mode, predictions=predictions, scaffold_fn=scaffold_fn)
    return output_spec

  return model_fn


def convert_examples_to_features(examples, seq_length, tokenizer):
  """Loads a data file into a list of `InputBatch`s."""

  features = []
  for (ex_index, example) in enumerate(examples):
    tokens_a = tokenizer.tokenize(example.text_a)

    tokens_b = None
    if example.text_b:
      tokens_b = tokenizer.tokenize(example.text_b)

    if tokens_b:
      # Modifies `tokens_a` and `tokens_b` in place so that the total
      # length is less than the specified length.
      # Account for [CLS], [SEP], [SEP] with "- 3"
      _truncate_seq_pair(tokens_a, tokens_b, seq_length - 3)
    else:
      # Account for [CLS] and [SEP] with "- 2"
      if len(tokens_a) > seq_length - 2:
        tokens_a = tokens_a[0:(seq_length - 2)]

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids: 0     0   0   0  0     0 0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = []
    input_type_ids = []
    tokens.append("[CLS]")
    input_type_ids.append(0)
    for token in tokens_a:
      tokens.append(token)
      input_type_ids.append(0)
    tokens.append("[SEP]")
    input_type_ids.append(0)

    if tokens_b:
      for token in tokens_b:
        tokens.append(token)
        input_type_ids.append(1)
      tokens.append("[SEP]")
      input_type_ids.append(1)
    
    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < seq_length:
      input_ids.append(0)
      input_mask.append(0)
      input_type_ids.append(0)

    assert len(input_ids) == seq_length
    assert len(input_mask) == seq_length
    assert len(input_type_ids) == seq_length

    if ex_index < 5:
      tf.logging.info("*** Example ***")
      tf.logging.info("unique_id: %s" % (example.unique_id))
      tf.logging.info("tokens: %s" % " ".join(
          [tokenization.printable_text(x) for x in tokens]))
      tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
      tf.logging.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
      tf.logging.info(
          "input_type_ids: %s" % " ".join([str(x) for x in input_type_ids]))

    features.append(
        InputFeatures(
            unique_id=example.unique_id,
            tokens=tokens,
            input_ids=input_ids,
            input_mask=input_mask,
            input_type_ids=input_type_ids))
  return features


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
  """Truncates a sequence pair in place to the maximum length."""

  # This is a simple heuristic which will always truncate the longer sequence
  # one token at a time. This makes more sense than truncating an equal percent
  # of tokens from each, since if one sequence is very short then each token
  # that's truncated likely contains more information than a longer sequence.
  while True:
    total_length = len(tokens_a) + len(tokens_b)
    if total_length <= max_length:
      break
    if len(tokens_a) > len(tokens_b):
      tokens_a.pop()
    else:
      tokens_b.pop()


def read_examples(input_file):
  """Read a list of `InputExample`s from an input file."""
  examples = []
  unique_id = 0
  with tf.gfile.GFile(input_file, "r") as reader:
    while True:
      line = tokenization.convert_to_unicode(reader.readline())
      if not line:
        break
      line = line.strip()
      text_a = None
      text_b = None
      m = re.match(r"^(.*) \|\|\| (.*)$", line)
      if m is None:
        text_a = line
      else:
        text_a = m.group(1)
        text_b = m.group(2)
      examples.append(
          InputExample(unique_id=unique_id, text_a=text_a, text_b=text_b))
      unique_id += 1
  return examples



In [ ]:
val_df = pd.read_table('gap-validation.tsv', index_col='ID').reset_index(drop=True)
test_df  = pd.read_table('gap-test.tsv', index_col='ID').reset_index(drop=True)
dev_df  = pd.read_table('gap-development.tsv', index_col='ID').reset_index(drop=True)

In [ ]:
tf.logging.set_verbosity(tf.logging.INFO)
layer_indexes = [-1]

bert_config = modeling.BertConfig.from_json_file('uncased_L-12_H-768_A-12/bert_config.json')
model_path = "uncased_L-12_H-768_A-12"
do_lower_case = True
tokenizer = tokenization.FullTokenizer(vocab_file='uncased_L-12_H-768_A-12/vocab.txt', do_lower_case=True)
is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
run_config = tf.contrib.tpu.RunConfig(
    master=None,
    tpu_config=tf.contrib.tpu.TPUConfig(
        num_shards=8,
        per_host_input_for_training=is_per_host))

text = val_df['Text']
text.to_csv('input.txt', index=False, header=False)
examples = read_examples('input.txt')

features = convert_examples_to_features(
    examples=examples, seq_length=512, tokenizer=tokenizer)

all_unique_ids = []
all_input_ids = []
all_input_mask = []
all_input_type_ids = []

for feature in features:
    all_unique_ids.append(feature.unique_id)
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_input_type_ids.append(feature.input_type_ids)

all_unique_ids = torch.tensor(all_unique_ids, dtype=torch.long) 
all_input_ids = torch.tensor(all_input_ids, dtype=torch.long) 
all_input_mask = torch.tensor(all_input_mask, dtype=torch.long) 
all_input_type_ids = torch.tensor(all_input_type_ids, dtype=torch.long) 

train_data = TensorDataset(all_unique_ids, all_input_ids, all_input_mask, all_input_type_ids)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:*** Example ***
INFO:tensorflow:unique_id: 0
INFO:tensorflow:tokens: [CLS] " he admitted making four trips to china and playing golf there . he also admitted that z ##te officials , whom he says are his golf buddies , hosted and paid for the trips . jose de ve ##ne ##cia iii , son of house speaker jose de ve ##ne ##cia jr , alleged that aba ##los offered him us $ 10 million to withdraw his proposal on the n ##bn project . " [SEP]
INFO:tensorflow:input_ids: 101 1000 2002 4914 2437 2176 9109 2000 2859 1998 2652 5439 2045 1012 2002 2036 4914 2008 1062 2618 4584 1010 3183 2002 2758 2024 2010 5439 24115 1010 4354 199

# Process the data to extract features and get word embeddings

In [ ]:
def count_char(text, offset):   
    # count = 0
    # for pos in range(offset):
    #     if text[pos] != " ": count +=1
    # return count
    return sum([1 if text[pos] != " " else 0 for pos in range(offset)])

def candidate_length(candidate):
    # count = 0
    # for i in range(len(candidate)):
    #     if candidate[i] !=  " ": count += 1
    # return count
    return sum([1 if candidate[i] !=  " " else 0 for i in range(len(candidate))])

def count_token_length_special(token):
    count = 0
    special_token = ["#", " "]
    # for i in range(len(token)):
    #     if token[i] not in special_token: count+=1
    # return count
    return sum([1 if token[i] not in special_token else 0 for i in range(len(token))])

def embed_by_bert(df, path_to_bert='uncased_L-12_H-768_A-12', embed_size=768, batch_size=8,
                 layers='-1', max_seq_length=256):
    
    text = df['Text']
    text.to_csv('input.txt', index=False, header=False)
    os.system(f"python3 extract_features.py \
               --input_file=input.txt \
               --output_file=output.jsonl \
               --vocab_file={path_to_bert}/vocab.txt \
               --bert_config_file={path_to_bert}/bert_config.json \
               --init_checkpoint={path_to_bert}/bert_model.ckpt \
               --layers={layers} \
               --max_seq_length={max_seq_length} \
               --batch_size={batch_size}")
    
    bert_output = pd.read_json("output.jsonl", lines=True)
    # print(bert_output.head())
    
    # os.system("rm input.txt")
    # os.system("rm output.jsonl")
    
    index = df.index
    columns = ["emb_A", "emb_B", "emb_P", "label"]
    emb = pd.DataFrame(index = index, columns = columns)
    emb.index.name = "ID"
    
    for i in tqdm(range(len(text))):
        
        features = bert_output.loc[i, "features"]
        P_char_start = count_char(df.loc[i, 'Text'], df.loc[i, 'Pronoun-offset'])
        A_char_start = count_char(df.loc[i, 'Text'], df.loc[i, 'A-offset'])
        B_char_start = count_char(df.loc[i, 'Text'], df.loc[i, 'B-offset'])
        A_length = candidate_length(df.loc[i, 'A'])
        B_length = candidate_length(df.loc[i, 'B'])
        
        emb_A, emb_B, emb_P = np.zeros(embed_size), np.zeros(embed_size), np.zeros(embed_size)
        char_count, cnt_A, cnt_B = 0, 0, 0
        
        for j in range(2, len(features)):
            token = features[j]["token"]
            token_length = count_token_length_special(token)
            if char_count == P_char_start:
                emb_P += np.asarray(features[j]["layers"][0]['values']) 
            if char_count in range(A_char_start, A_char_start + A_length):
                emb_A += np.asarray(features[j]["layers"][0]['values'])
                cnt_A += 1
            if char_count in range(B_char_start, B_char_start + B_length):
                emb_B += np.asarray(features[j]["layers"][0]['values'])
                cnt_B += 1                
            char_count += token_length
        
        if cnt_A > 0:
            emb_A /= cnt_A
        if cnt_B > 0:
            emb_B /= cnt_B
        
        label = "Neither"
        if (df.loc[i,"A-coref"] == True):
            label = "A"
        if (df.loc[i,"B-coref"] == True):
            label = "B"

        emb.iloc[i] = [emb_A, emb_B, emb_P, label]
        
    return emb    

# Multi-class Neural Network 
## Detailed explaination of how to train a multiclass neural network model using PyTorch: https://towardsdatascience.com/pytorch-tabular-multiclass-classification-9f8211a123ab

In [ ]:
class MulticlassClassification(nn.Module):
    def __init__(self, num_feature, num_class):
        super(MulticlassClassification, self).__init__()
        
        self.layer_1 = nn.Linear(num_feature, 512)
        self.layer_2 = nn.Linear(512, 128)
        self.layer_3 = nn.Linear(128, 64)
        self.layer_out = nn.Linear(64, num_class) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)
        self.batchnorm1 = nn.BatchNorm1d(512)
        self.batchnorm2 = nn.BatchNorm1d(128)
        self.batchnorm3 = nn.BatchNorm1d(64)
        
    def forward(self, x):
        x = self.layer_1(x)
        x = self.batchnorm1(x)
        x = self.relu(x)
        
        x = self.layer_2(x)
        x = self.batchnorm2(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_3(x)
        x = self.batchnorm3(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_out(x)
        
        return x

In [ ]:
def featurize(embedding_df):
    
    pronoun_embs, a_embs, b_embs, labels = [], [], [], []
    
    for i in tqdm(range(len(embedding_df))):
        
        pronoun_embs.append(embedding_df.loc[i, "emb_P"])
        a_embs.append(embedding_df.loc[i, "emb_A"])
        b_embs.append(embedding_df.loc[i, "emb_B"])

        label_map = {'A': 0, 'B': 1, 'Neither': 2}
        labels.append(label_map[embedding_df.loc[i, "label"]])

    
    a_embs = np.asarray(a_embs).astype('float')
    b_embs = np.asarray(b_embs).astype('float') 
    pronoun_embs = np.asarray(pronoun_embs).astype('float')
    
    return np.concatenate([a_embs, b_embs, pronoun_embs], axis=1), np.asarray(labels)

In [ ]:
class ClassifierDataset(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)


In [ ]:
def get_class_distribution(obj):
    count_dict = {
        "A": 0,
        "B": 0,
        "N": 0,
    }
    
    for i in obj:
        if i == 0: 
            count_dict['A'] += 1
        elif i == 1: 
            count_dict['B'] += 1
        elif i == 2: 
            count_dict['N'] += 1
            
    return count_dict

In [ ]:
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
    
    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)
    
    acc = torch.round(acc * 100)
    
    return acc

# Training the model based on the embeddings from ALBERT.

In [ ]:
import time
start = time.time()

val_df = pd.read_table('gap-validation.tsv', index_col='ID').reset_index(drop=True)
test_df  = pd.read_table('gap-test.tsv', index_col='ID').reset_index(drop=True)
dev_df  = pd.read_table('gap-development.tsv', index_col='ID').reset_index(drop=True)

dev_bert_emb = embed_by_bert(dev_df)
val_bert_emb = embed_by_bert(val_df)
test_bert_emb = embed_by_bert(test_df)

X_train, y_train = featurize(dev_bert_emb)
X_val , y_val = featurize(val_bert_emb)
X_test, y_test = featurize(test_bert_emb)

train_dataset = ClassifierDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).long())
val_dataset = ClassifierDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).long())
test_dataset = ClassifierDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).long())

target_list = []
for _, t in train_dataset:
    target_list.append(t)
    
target_list = torch.tensor(target_list)
target_list = target_list[torch.randperm(len(target_list))]

class_count = [i for i in get_class_distribution(y_train).values()]
class_weights = 1./torch.tensor(class_count, dtype=torch.float) 
# print(class_weights)

class_weights_all = class_weights[target_list]

weighted_sampler = WeightedRandomSampler(
    weights=class_weights_all,
    num_samples=len(class_weights_all),
    replacement=True
)

EPOCHS = 300
BATCH_SIZE = 16
LEARNING_RATE = 0.0007
NUM_FEATURES = 2304
NUM_CLASSES = 3

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, sampler=weighted_sampler)
val_loader = DataLoader(dataset=val_dataset, batch_size=1)
test_loader = DataLoader(dataset=test_dataset, batch_size=1)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(device)

model = MulticlassClassification(num_feature = NUM_FEATURES, num_class=NUM_CLASSES)
model.to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
# print(model)

accuracy_stats = {
    'train': [],
    "val": []
}
loss_stats = {
    'train': [],
    "val": []
}

bert_train_preds = []

print("Begin training.")
for e in tqdm(range(1, EPOCHS+1)):
    
    # TRAINING
    train_epoch_loss = 0
    train_epoch_acc = 0
    model.train()
    for X_train_batch, y_train_batch in train_loader:
        X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
        optimizer.zero_grad()
        
        y_train_pred = model(X_train_batch)

        #alt
        probs = nn.Softmax(dim=1)(y_train_pred)
        # preds = torch.max(probs, 1)[1]
        bert_train_preds.append([float(probs[0][0]), float(probs[0][1]), float(probs[0][2])])

        train_loss = criterion(y_train_pred, y_train_batch)
        train_acc = multi_acc(y_train_pred, y_train_batch)
        
        train_loss.backward()
        optimizer.step()
        
        train_epoch_loss += train_loss.item()
        train_epoch_acc += train_acc.item()
        
        
    # VALIDATION    
    with torch.no_grad():
        
        val_epoch_loss = 0
        val_epoch_acc = 0
        
        model.eval()
        for X_val_batch, y_val_batch in val_loader:
            X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            
            y_val_pred = model(X_val_batch)
                        
            val_loss = criterion(y_val_pred, y_val_batch)
            val_acc = multi_acc(y_val_pred, y_val_batch)
            
            val_epoch_loss += val_loss.item()
            val_epoch_acc += val_acc.item()
    loss_stats['train'].append(train_epoch_loss/len(train_loader))
    loss_stats['val'].append(val_epoch_loss/len(val_loader))
    accuracy_stats['train'].append(train_epoch_acc/len(train_loader))
    accuracy_stats['val'].append(val_epoch_acc/len(val_loader))
                              
    
    print(f'Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.5f}') 
    print(f'Val Loss: {val_epoch_loss/len(val_loader):.5f}')
    print(f'Train Acc: {train_epoch_acc/len(train_loader):.3f}')
    print(f'Val Acc: {val_epoch_acc/len(val_loader):.3f}')

end = time.time()
print(end - start)

/usr/local/lib/python3.7/dist-packages/pandas/core/internals/blocks.py:849: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr_value = np.array(value)


/usr/local/lib/python3.7/dist-packages/pandas/core/internals/blocks.py:849: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr_value = np.array(value)


/usr/local/lib/python3.7/dist-packages/pandas/core/internals/blocks.py:849: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr_value = np.array(value)



Begin training.


Epoch 001: | Train Loss: 0.72234
Val Loss: 0.86088
Train Acc: 70.080
Val Acc: 60.352
Epoch 002: | Train Loss: 0.49479
Val Loss: 0.73710
Train Acc: 82.080
Val Acc: 68.502
Epoch 003: | Train Loss: 0.36685
Val Loss: 0.72513
Train Acc: 87.856
Val Acc: 70.485
Epoch 004: | Train Loss: 0.31318
Val Loss: 0.68824
Train Acc: 89.600
Val Acc: 73.128
Epoch 005: | Train Loss: 0.24478
Val Loss: 0.69549
Train Acc: 92.088
Val Acc: 73.348
Epoch 006: | Train Loss: 0.17154
Val Loss: 0.77572
Train Acc: 94.656
Val Acc: 71.806
Epoch 007: | Train Loss: 0.18551
Val Loss: 0.83528
Train Acc: 94.616
Val Acc: 73.568
Epoch 008: | Train Loss: 0.16419
Val Loss: 0.74725
Train Acc: 95.184
Val Acc: 74.890
Epoch 009: | Train Loss: 0.11376
Val Loss: 0.81310
Train Acc: 96.120
Val Acc: 72.026
Epoch 010: | Train Loss: 0.15295
Val Loss: 0.79284
Train Acc: 94.136
Val Acc: 75.330
Epoch 011: | Train Loss: 0.08650
Val Loss: 0.78209
Train Acc: 97.352
Val Acc: 73.568
Epoch 012: | Train Loss: 0.08201
Val Loss: 0.80794
Train Acc: 97.

In [ ]:
print(end - start)

736.0356187820435


# Get Training Weights for Late fusion Model

In [ ]:
y_train_pred_list = []
bert_train_preds = []
with torch.no_grad():
    model.eval()
    for X_batch, _ in train_loader:
        X_batch = X_batch.to(device)
        y_train_pred = model(X_batch)
        bert_train_preds.extend(y_train_pred.tolist())
        _, y_pred_tags = torch.max(y_train_pred, dim = 1)
        y_train_pred_list.append(y_pred_tags.cpu().numpy())
y_train_pred_list = [a.squeeze().tolist() for a in y_train_pred_list]

In [ ]:
print(bert_train_preds)
print(len(bert_train_preds))

[[10.974891662597656, -15.2702054977417, -9.919142723083496], [-2.1368441581726074, 5.622532844543457, -10.146480560302734], [2.823481559753418, -5.436559677124023, -3.65732741355896], [0.7114617824554443, 1.380729079246521, -8.498754501342773], [-16.189619064331055, 11.740434646606445, -7.50857400894165], [14.233391761779785, -30.41864013671875, -6.04236364364624], [8.16143798828125, -13.901387214660645, -6.591201305389404], [-13.780210494995117, 8.5954008102417, -4.933476448059082], [8.792167663574219, -10.907105445861816, -9.330320358276367], [-14.780328750610352, 10.777530670166016, -7.564312934875488], [-18.40755844116211, 12.96097183227539, -7.824159145355225], [-9.515655517578125, -8.732131004333496, 7.551417827606201], [4.703126430511475, -2.1226806640625, -10.19816780090332], [11.80275821685791, -21.412086486816406, -7.666904449462891], [-15.597121238708496, 11.834696769714355, -7.664235591888428], [-10.238750457763672, 10.412125587463379, -9.43939208984375], [-10.742204666137

# Get predictions for calculating accuracy, precision, recall

In [ ]:
y_pred_list = []
bert_preds = []
with torch.no_grad():
    model.eval()
    for X_batch, _ in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        bert_preds.append(y_test_pred)
        _, y_pred_tags = torch.max(y_test_pred, dim = 1)
        y_pred_list.append(y_pred_tags.cpu().numpy())
y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

In [ ]:
print(y_pred_list)

[1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 2, 0, 1, 2, 1, 0, 1, 1, 0, 1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 2, 0, 1, 1, 1, 0, 2, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 2, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 2, 2, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 2, 0, 2, 1, 1, 1, 1, 1, 2, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 2, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 0, 0, 1, 1, 0, 0, 2, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 2, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 

In [ ]:
print(y_test)

[1 0 0 ... 0 0 0]


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_list))

              precision    recall  f1-score   support

           0       0.81      0.80      0.80       918
           1       0.73      0.84      0.78       855
           2       0.69      0.35      0.46       227

    accuracy                           0.76      2000
   macro avg       0.74      0.66      0.68      2000
weighted avg       0.76      0.76      0.76      2000



In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred_list))

[[731 143  44]
 [110 710  35]
 [ 48  83  96]]


In [ ]:
y_pred_list = []
bert_test_preds = []
with torch.no_grad():
    model.eval()
    for X_batch, _ in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        # probs = nn.Softmax(dim=1)(y_test_pred)
        # preds = torch.max(probs, 1)[1]
        bert_test_preds.extend(y_test_pred.tolist())
        _, y_pred_tags = torch.max(y_test_pred, dim = 1)
        y_pred_list.append(y_pred_tags.cpu().numpy())
y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

In [ ]:
print(bert_test_preds)
print(len(bert_test_preds))

[[-0.24238795042037964, 3.2520651817321777, -9.946405410766602], [1.8526924848556519, 0.5240638256072998, -9.218080520629883], [7.4488420486450195, -14.324432373046875, -4.955216407775879], [-3.606236457824707, 4.643353462219238, -6.371252059936523], [11.284456253051758, -20.00550079345703, -7.712418079376221], [-1.8777062892913818, 2.55039119720459, -5.070782661437988], [5.708907604217529, -2.765002727508545, -13.647380828857422], [-6.833441257476807, 2.861372947692871, -2.2818148136138916], [3.734518527984619, -5.66752815246582, -4.643352031707764], [-16.75071144104004, 11.412686347961426, -6.937999248504639], [-17.511507034301758, 14.834332466125488, -10.879517555236816], [-3.7808899879455566, -13.296448707580566, 6.791179656982422], [-11.2894926071167, 7.714671611785889, -5.147165775299072], [-27.494136810302734, 20.222579956054688, -11.74782657623291], [6.418975830078125, -6.168317794799805, -8.51092529296875], [3.9058966636657715, -0.8490347862243652, -10.590173721313477], [16.20

# Save weights which can be later used for Late Fusion

In [ ]:
import csv
with open('bert_train', 'w') as f:
    # using csv.writer method from CSV package
    write = csv.writer(f)   
    write.writerows(bert_train_preds)

In [ ]:
import csv
with open('bert_test', 'w') as f:
    # using csv.writer method from CSV package
    write = csv.writer(f)   
    write.writerows(bert_test_preds)

# Prediction results to use in Project

In [ ]:
with torch.no_grad():
    model.eval()
    for X_batch, c in test_loader:
        X_batch = X_batch.to(device)
        print(c)
        y_test_pred = model(X_batch)
        probs = nn.Softmax(dim=1)(y_test_pred)
        preds = torch.max(probs, 1)[1]
        print(probs)
        print(preds)
        # print(y_test_pred)
        _, y_pred_tags = torch.max(y_test_pred, dim = 1)
        # print(_)
        # print(y_pred_tags)
        y_pred_list.append(y_pred_tags.cpu().numpy())
        break

tensor([1])
tensor([[1.3589e-03, 9.9864e-01, 4.2751e-08]], device='cuda:0')
tensor([1], device='cuda:0')


In [ ]:
# ind = 1636, 1656
ind = 1656
print(test_df.iloc[ind]["Text"])
off = test_df.iloc[ind]["Pronoun-offset"]
print(test_df.iloc[ind]["Text"][off-7:off+ 10])
print(test_df.iloc[ind])

Jon Dolan from Rolling Stone gave the album three out of five stars, writing that ``Perry and her longtime collaborators Dr. Luke and Max Martin often go for a darker, moodier intimacy * la high-end Swedish divas Robyn and Lykke Li. Perry has always done a great job of letting us know she's in on the joke of pop stardom.
s know she's in o
Text              Jon Dolan from Rolling Stone gave the album th...
Pronoun                                                         she
Pronoun-offset                                                  286
A                                                             Robyn
A-offset                                                        213
A-coref                                                       False
B                                                          Lykke Li
B-offset                                                        223
B-coref                                                       False
URL               http://en.wikipedia.org/wiki/

Gender bias has been a significant issue in the latest years, apparent in
all fields of Computational Linguistics, spanning from text representation approaches to Coreference Resolution. In Coreference Resolution the bias appeared favouring the Masculine over Feminine predictions. As the bias originated in the datasets, the need to have unbiased training data resulted in the
introduction of new sources

To enable
a more targeted approach, Mind the GAP (Webster et al., 2018) was published
which aimed towards Gender Ambiguous Pronoun identification. As a result, it
was recently used by Kaggle’s Gender Pronoun Resolution competition5
, and it
is considered to be the benchmark for the task.

In [ ]:
ind = 1656
print(test_df.iloc[ind]["Text"])
off = test_df.iloc[ind]["Pronoun-offset"]
print(test_df.iloc[ind]["Text"][off-7:off+ 10])
print(test_df.iloc[ind])

Jon Dolan from Rolling Stone gave the album three out of five stars, writing that ``Perry and her longtime collaborators Dr. Luke and Max Martin often go for a darker, moodier intimacy * la high-end Swedish divas Robyn and Lykke Li. Perry has always done a great job of letting us know she's in on the joke of pop stardom.
s know she's in o
Text              Jon Dolan from Rolling Stone gave the album th...
Pronoun                                                         she
Pronoun-offset                                                  286
A                                                             Robyn
A-offset                                                        213
A-coref                                                       False
B                                                          Lykke Li
B-offset                                                        223
B-coref                                                       False
URL               http://en.wikipedia.org/wiki/

In [ ]:
with torch.no_grad():
    model.eval()
    for i, data in enumerate(test_loader, 0):
        if i == ind:
            x, y = data 
            x = x.to(device)
            y_test_pred = model(x)
            probs = nn.Softmax(dim=1)(y_test_pred)
            preds = torch.max(probs, 1)[1]
            print(probs)
            print(round(float(probs[0][0]),4), round(float(probs[0][1]),4), round(float(probs[0][2]),4))
            print(preds)
            break



tensor([[7.4315e-07, 2.9790e-04, 9.9970e-01]], device='cuda:0')
0.0 0.0003 0.9997
tensor([2], device='cuda:0')
